In [2]:
!pip install evaluate dataset scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.0 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.42
    Uninstalling SQLAlchemy-2.0.42:
      Successfully uninstalled SQLAlchemy-2.0.42
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [4]:
# main.py
# Ensure you have the necessary libraries installed:
# pip install transformers torch peft accelerate datasets evaluate


import torch
import numpy as np
import evaluate
import logging
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer


# --- 1. Setup and Configuration ---
def setup_logging():
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[logging.StreamHandler()]
    )
    return logging.getLogger(__name__)

logger = setup_logging()

base_model_id = "gpt2"
dataset_name = "rotten_tomatoes"
lora_adapter_dir = "gpt2-lora-rotten-tomatoes"

def get_device():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    logger.info(f"Using device: {device}")
    return device


def load_tokenizer_and_dataset(base_model_id, dataset_name):
    logger.info("Step 2: Loading tokenizer and dataset...")
    tokenizer = AutoTokenizer.from_pretrained(base_model_id)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    dataset = load_dataset(dataset_name)
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)
    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
    small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
    logger.info("Tokenizer and dataset loaded and preprocessed.")
    return tokenizer, small_train_dataset, small_eval_dataset



def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def load_and_evaluate_base_model(base_model_id, tokenizer, eval_dataset, device):
    logger.info("\nStep 3: Loading and evaluating the original foundation model...")
    model = AutoModelForSequenceClassification.from_pretrained(
        base_model_id,
        num_labels=2,
        pad_token_id=tokenizer.pad_token_id
    )
    model.to(device)
    eval_trainer = Trainer(
        model=model,
        args=TrainingArguments(output_dir="./eval_results", per_device_eval_batch_size=8), # Removed unsupported arguments for compatibility
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )
    results = eval_trainer.evaluate()
    logger.info(f"Original model accuracy: {results['eval_accuracy']:.4f}")
    return model, results



def peft_fine_tune(model, train_dataset, eval_dataset, lora_adapter_dir, device):
    logger.info("\nStep 4: Setting up and performing PEFT with LoRA...")
    config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["c_attn"],
        lora_dropout=0.05,
        bias="none",
        task_type="SEQ_CLS"
    )
    lora_model = get_peft_model(model, config)
    lora_model.to(device)
    logger.info("PEFT model with LoRA created.")
    lora_model.print_trainable_parameters()
    training_args = TrainingArguments(
        output_dir=lora_adapter_dir,
        num_train_epochs=1,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        logging_dir=f"{lora_adapter_dir}/logs",
        logging_steps=10,
        # report_to="none" # Removed unsupported arguments for compatibility
    )
    trainer = Trainer(
        model=lora_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )
    logger.info("Starting fine-tuning...")
    trainer.train()
    logger.info("Fine-tuning complete.")
    trainer.save_model(lora_adapter_dir)
    logger.info(f"Trained LoRA adapter saved to '{lora_adapter_dir}'.")
    return lora_model, trainer



def evaluate_fine_tuned_model(lora_adapter_dir, eval_dataset, device):
    logger.info("\nStep 5: Loading and evaluating the fine-tuned PEFT model...")
    # Load fine-tuned model and ensure pad_token is defined for GPT2
    fine_tuned_model = AutoPeftModelForSequenceClassification.from_pretrained(
        lora_adapter_dir,
    )
    # Load tokenizer to set padding token
    tokenizer = AutoTokenizer.from_pretrained(base_model_id)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    fine_tuned_model.config.pad_token_id = tokenizer.pad_token_id
    fine_tuned_model.to(device)
    tuned_trainer = Trainer(
        model=fine_tuned_model,
        args=TrainingArguments(output_dir="./tuned_eval_results", per_device_eval_batch_size=8), # Removed unsupported arguments for compatibility
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )
    results = tuned_trainer.evaluate()
    return results



def main():
    device = get_device()
    tokenizer, small_train_dataset, small_eval_dataset = load_tokenizer_and_dataset(base_model_id, dataset_name)
    original_model, original_model_results = load_and_evaluate_base_model(base_model_id, tokenizer, small_eval_dataset, device)
    lora_model, trainer = peft_fine_tune(original_model, small_train_dataset, small_eval_dataset, lora_adapter_dir, device)
    # Free up memory
    del original_model, lora_model, trainer
    torch.cuda.empty_cache()
    tuned_model_results = evaluate_fine_tuned_model(lora_adapter_dir, small_eval_dataset, device)
    logger.info("\n--- Performance Comparison ---")
    logger.info(f"Original Model Accuracy: {original_model_results['eval_accuracy']:.4f}")
    logger.info(f"Fine-Tuned Model Accuracy: {tuned_model_results['eval_accuracy']:.4f}")
    print("\n--- Performance Comparison ---")
    print(f"Original Model Accuracy: {original_model_results['eval_accuracy']:.4f}")
    print(f"Fine-Tuned Model Accuracy: {tuned_model_results['eval_accuracy']:.4f}")
    print("----------------------------")

if __name__ == "__main__":
    main()

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1803: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 591,360 || all params: 125,032,704 || trainable%: 0.4730


Step,Training Loss
10,0.692600
20,0.689500
30,0.677300
40,0.685000
50,0.671100
60,0.683500
70,0.679700
80,0.698900
90,0.680500
100,0.697000


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



--- Performance Comparison ---
Original Model Accuracy: 0.4990
Fine-Tuned Model Accuracy: 0.5280
----------------------------
